In [17]:
import torch # torch will allow us to create tensors.

import pytorch_lightning as L # lightning has tons of cool tools that make neural networks easier
from torch.utils.data import TensorDataset, DataLoader # these are needed for the training data
from dataset.dataset import load_dataset
from module.crowd_layer import CrowdLayerModule
from module.ground_truth import ClassifierModule
from sklearn.metrics import accuracy_score

In [2]:
X_train, y_train, y_train_true, X_valid, y_valid_true, X_test, y_test_true = load_dataset()
X_train_tensor = torch.tensor(X_train).float()
y_train_tensor = torch.tensor(y_train).long()
y_train_true_tensor = torch.tensor(y_train_true).long()
X_valid_tensor = torch.tensor(X_valid).float()
y_valid_tensor = torch.tensor(y_valid_true).long()
X_test_tensor = torch.tensor(X_test).float()
y_test_tensor = torch.tensor(y_test_true).long()

In [3]:
batch_size = 64
n_classes = 10
dropout = 0.0
n_annotators = 44
n_epochs = 200

In [4]:
train_ds = TensorDataset(X_train_tensor, y_train_tensor)
train_dl = DataLoader(train_ds, batch_size=batch_size)
valid_ds = TensorDataset(X_valid_tensor, y_valid_tensor)
test_ds = TensorDataset(X_test_tensor, y_test_tensor)

In [5]:
lr = 0.01
weight_decay = 0.0001

In [6]:
gt_model = ClassifierModule(n_classes=n_classes, dropout=dropout)
cl_model = CrowdLayerModule(n_classes=n_classes, n_annotators=n_annotators, gt_net=gt_model, lr=lr, weight_decay=weight_decay)

Global seed set to 42
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'gt_net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['gt_net'])`.
  rank_zero_warn(


In [7]:
trainer = L.Trainer(max_epochs=n_epochs)
trainer.fit(cl_model, train_dataloaders=train_dl)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:200: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytorch_lightning/trainer/configuration_validator.py:108: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name             | Type             | Params
------------------------------------------------------
0 | gt_net           | ClassifierModule | 17.5 K
1 | annotator_layers | ModuleList       | 4.4 K 
------------------------------------------------------
21.9 K    Trainable params
0         Non-trainable params
21.9 K    Total 

Training: 0it [00:00, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

`Trainer.fit` stopped: `max_epochs=200` reached.


In [12]:
p_class, p_annotation = cl_model(X_test_tensor)

In [15]:
y_pred = p_class.argmax(dim=1)

In [18]:
accuracy_score(y_pred, y_test_true)

0.584